In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pandas.io import data, wb
import datetime
import os.path
import math

from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn import cross_validation
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier

from sklearn import metrics
    
from sklearn.externals import joblib

import talib as ta
from pandas.tseries.offsets import BDay


In [20]:
#def predict(symbol):
symbol = 'spy'
df_train = pd.read_csv('data/'+symbol+'-ohlc.csv', index_col='Date', parse_dates=True)
if (os.path.isfile('data/'+symbol+'-dividends.csv')):
    df_dividends = pd.read_csv('data/'+symbol+'-dividends.csv', index_col='Date', parse_dates=True)
    df_train = df_train.join(df_dividends)
    df_train['Dividends'].fillna(0, inplace=True)
else:
    df_train['Dividends']=0


pma10 = df_train['Close'].rolling(window=10,center=False).mean() + df_train['Dividends'].rolling(window=10, center=False).mean()
pma20 = df_train['Close'].rolling(window=20,center=False).mean() + df_train['Dividends'].rolling(window=20, center=False).mean()
pma50 = df_train['Close'].rolling(window=50,center=False).mean() + df_train['Dividends'].rolling(window=50, center=False).mean()
pma200 = df_train['Close'].rolling(window=200,center=False).mean() + df_train['Dividends'].rolling(window=200, center=False).mean()

 #distance from MAs
diffma10  = (df_train['Close']/pma10 -1)*100
diffma20 = (df_train['Close']/pma20 -1)*100
diffma50 = (df_train['Close']/pma50 -1)*100
diffma200 = (df_train['Close']/pma200 -1)*100

rmax5 = (df_train['High'].rolling(window=5,center=False).max()/df_train['Close'] - 1)*100
rmax10 = (df_train['High'].rolling(window=10,center=False).max()/df_train['Close'] -1)*100

rmin5 = (df_train['Close']/df_train['Low'].rolling(window=5,center=False).min() - 1)*100
rmin10 = (df_train['Close']/df_train['Low'].rolling(window=10,center=False).min() -1)*100

rmaxf5 = (df_train['High'].rolling(window=5,center=False).max()/df_train['Close'].shift(5) -1)*100
rmaxf10 = (df_train['High'].rolling(window=10,center=False).max()/df_train['Close'].shift(5) -1)*100

#sequence of higher high/higher low
hh = df_train['High'][1:] + df_train['Dividends'][1:] > df_train['High'][:-1] 
hl = df_train['Low'][1:] + df_train['Dividends'][1:] > df_train['Low'][:-1] 

#rolling average of hh and hl
hh10 = hh.rolling(window=10, center=False).mean()
hl10 = hl.rolling(window=10, center=False).mean()

#rsi
adjclose = df_train['Adj Close'].values
high = adjclose = df_train['High'].values
low = adjclose = df_train['Low'].values
close = adjclose = df_train['Close'].values
rsi = ta.RSI(adjclose, timeperiod=10)/100

#volume indicator
vma50 = df_train['Volume'].rolling(window=50, center=False).mean()
nvx1 = df_train['Volume']/vma50
nvma5 = df_train['Volume'].rolling(window=5, center=False).mean()/vma50
nvma10 = df_train['Volume'].rolling(window=10, center=False).mean()/vma50


#distance from MAs
df_train = df_train.assign(diffma10=diffma10)
df_train = df_train.assign(diffma20=diffma20)
df_train = df_train.assign(diffma50=diffma50)
df_train = df_train.assign(diffma200=diffma200)

df_train = df_train.assign(rmax5=rmax5)
df_train = df_train.assign(rmax10=rmax10)
df_train = df_train.assign(rmin5=rmin5)
df_train = df_train.assign(rmin10=rmin10)

df_train = df_train.assign(hh10=hh10)
df_train = df_train.assign(hl10=hl10)

df_train = df_train.assign(rsi=rsi)

#last day volume
df_train = df_train.assign(nvx1=nvx1)
df_train = df_train.assign(nvma5=nvma5)
df_train = df_train.assign(nvma10=nvma10)


df_train = df_train.assign(rmaxf5=rmaxf5)
df_train = df_train.assign(rmaxf10=rmaxf10)


y_train = df_train['rmaxf5'].shift(-5).values[200:-11]



atr = ta.ATR(high, low, close, timeperiod=10)[-1]
atr = '%.2f' % round(atr, 2)
df_train.to_csv('data/'+symbol+'-train.csv')
remove = ['Open','High','Low','Close','Volume','Adj Close','Dividends', 'rmaxf5','rmaxf10']
#df_train = df_train.drop(remove, axis=1)

X_train = df_train.values[200:-11, :]
X_test = df_train.values[-11:,:]
rowCount = len(X_train)

#df_train.describe()
df_train.corr()

#for rmaxf5, shift 5 position back to get future value for rmaxf5
#model, rmax_score  = train(X_train, y_train)
#y_test_pred = model.predict_proba(X_test);
#prob_rmax = y_test_pred[-1,:]    
#print(prob_rmax)

    
    #return


,Open,High,Low,Close,Volume,Adj Close,Dividends,diffma10,diffma20,diffma50,...,rmin5,rmin10,hh10,hl10,rsi,nvx1,nvma5,nvma10,rmaxf5,rmaxf10
Open,1.000000,0.999825,0.999776,0.999615,0.286452,0.980111,0.043076,0.007419,0.013236,0.022593,...,-0.036720,-0.034057,0.049526,0.058531,-0.003789,-0.047468,-0.069916,-0.077004,-0.032903,-0.051127
High,0.999825,1.000000,0.999651,0.999774,0.291501,0.979923,0.042506,0.008086,0.011384,0.018349,...,-0.028081,-0.027205,0.046801,0.055412,-0.005278,-0.043936,-0.066424,-0.073741,-0.029336,-0.045189
Low,0.999776,0.999651,1.000000,0.999789,0.279834,0.980772,0.043166,0.018034,0.023316,0.032149,...,-0.036168,-0.033552,0.054594,0.064383,0.006184,-0.054208,-0.074419,-0.080779,-0.034098,-0.055755
Close,0.999615,0.999774,0.999789,1.000000,0.285276,0.980602,0.042199,0.019773,0.022297,0.028213,...,-0.025213,-0.024922,0.051629,0.060893,0.005334,-0.049866,-0.070612,-0.077334,-0.030700,-0.049526
Volume,0.286452,0.291501,0.279834,0.285276,1.000000,0.359236,0.044835,-0.225887,-0.276632,-0.334440,...,0.178964,0.145224,-0.148357,-0.158316,-0.227724,0.178999,0.132928,0.104975,0.109113,0.260170
Adj Close,0.980111,0.979923,0.980772,0.980602,0.359236,1.000000,0.047041,0.015137,0.016858,0.019981,...,-0.038806,-0.038583,0.059546,0.066787,0.011650,-0.051555,-0.073544,-0.080968,-0.046136,-0.063565
Dividends,0.043076,0.042506,0.043166,0.042199,0.044835,0.047041,1.000000,-0.013267,-0.004256,-0.001061,...,-0.018751,-0.013867,-0.002202,0.014705,-0.007414,0.028486,0.021848,0.006575,0.014367,0.003989
diffma10,0.007419,0.008086,0.018034,0.019773,-0.225887,0.015137,-0.013267,1.000000,0.875732,0.610891,...,0.478435,0.618854,0.522398,0.527289,0.773152,-0.328662,-0.316374,-0.197381,0.498387,0.133204
diffma20,0.013236,0.011384,0.023316,0.022297,-0.276632,0.016858,-0.004256,0.875732,1.000000,0.813858,...,0.215441,0.415448,0.632255,0.628074,0.854538,-0.352345,-0.435683,-0.399249,0.311886,-0.120169
diffma50,0.022593,0.018349,0.032149,0.028213,-0.334440,0.019981,-0.001061,0.610891,0.813858,1.000000,...,-0.070858,0.058153,0.525495,0.490692,0.769893,-0.289200,-0.402424,-0.437313,0.043591,-0.327690


In [15]:
def train(X_train, y_train):  

    model = RandomForestClassifier(n_estimators=350)     #best so far
    #model = AdaBoostClassifier(n_estimators=100)
    #model = LogisticRegression(random_state=1)
    #model = GradientBoostingClassifier(n_estimators=300)
    #model = KNeighborsClassifier(n_neighbors=15)

    
    scores = cross_validation.cross_val_score(model, X_train, y_train, cv=5, scoring='precision')
    mean = scores.mean()
    std = scores.std()
    scoreStr = '%.2f' % round(mean, 2) +"/"+'%.2f' % round(std, 2)
    
    model.fit(X_train, y_train)
    #print(model.feature_importances_)
    return model,scoreStr

In [16]:
predict('SPY')

NameError: name 'predict' is not defined

In [ ]:

    

    




    #for hl
    y_train = df_train['hl'].shift(-1).values[200:-11].astype(bool)        
    model, hl_score = train(X_train, y_train)
    #save_model('hl', symbol)
    y_test_pred = model.predict_proba(X_test);
    prob_hl = y_test_pred[-1,1]
    #predict score on the most recent 10 period(out of sample test)
    recent_hl_score = sum(df_train['hl'].values[-10:].astype(bool) == (y_test_pred[:, 1]>0.5)[:10])
    
    pdate = df_train.index[-1] + BDay(1)

    return pdate, atr, prob_hh, prob_hl, hh_score, hl_score, recent_hh_score, recent_hl_score, rowCount

In [ ]:
def save_model(target, symbol):
    model_dir = 'models/predict-'+target+'-'+symbol
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    joblib.dump(model, f_name) 

In [ ]:
def get_type(prob_hh, prob_hl):
    ptype = '..'
    if(prob_hh > 0.5):
        if(prob_hl < 0.5 ):
            ptype = '***'
        else:
            ptype = '++'
    else:
        if(prob_hl < 0.5):
            ptype = '--'
            
    return ptype

In [ ]:
#symbols = ['spy', 'qqq','iwm', 'eem', 'aapl', 'ibm', 'fb','goog', 'amzn','gs','intc', 'dva','noah', 'lc', 'renn', 'tsla', 'hog','600519.SS']
symbols = ['spy']
params = ['atr', 'prob_hh', 'prob_hl', 'hh_score', 'hl_score','rhh_score', 'rhl_score', 'rowCount', 'pdate', 'type']
df_predicts = pd.DataFrame(columns=symbols, index=params)

for symbol in symbols:
    print('\npredicting ' + symbol)
    pdate, atr, prob_hh, prob_hl, hh_score, hl_score, recent_hh_score, recent_hl_score, rowCount = predict(symbol)
    pdateStr = pdate.strftime("%Y-%m-%d")
    ptype = get_type(prob_hh, prob_hl)
    df_predicts[symbol] = (atr, '%.2f' % round(prob_hh, 2), '%.2f' % round(prob_hl, 2), 
                           hh_score, hl_score, recent_hh_score, recent_hl_score, rowCount, pdateStr, ptype)
    print(symbol, atr, 'hh=%.2f' % round(prob_hh, 2), 'hl=%.2f' % round(prob_hl, 2), 
          hh_score, hl_score, recent_hh_score, recent_hl_score, rowCount, pdateStr, ptype)

df_predicts.to_csv('predictions-'+pdateStr+'.csv')
print("Prediction for "+pdateStr+" completed.")

In [ ]:
def e_predict(X_train, y_train, X_test):

    clf1 = LogisticRegression(random_state=1)
    clf2 = RandomForestClassifier(n_estimators=50)
    clf3 = GaussianNB()
    clf4 = GradientBoostingClassifier(n_estimators=100)
    clf5 = KNeighborsClassifier(n_neighbors=15)
    
    eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3), 
                                        ('gb', clf4), ('knn',clf5)], voting='soft')

    for clf in [clf1, clf2, clf3, clf4, clf5, eclf]: 
        print(type(clf).__name__, "\n")
        scores = cross_validation.cross_val_score(clf, X_train, y_train, cv=5, scoring='precision')
        print("Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
        clf.fit(X_train, y_train)
        y_test_pred = clf.predict_proba(X_test);
        return y_test_pred
